In [ ]:
````xml
<VSCode.Cell language="markdown">
<a href="https://colab.research.google.com/github/madhusudhanrao-ppm/oracle-ai-developer-hub/blob/main/notebooks/multicloud-oracledb-at-google/create-ai-agent-memory-google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</VSCode.Cell>
<VSCode.Cell language="markdown">
# Oracle Database AI Agent Memory on Google Cloud Platform

This notebook demonstrates how to use **Oracle Autonomous Database hosted on Google Cloud Platform** as the persistent memory backend for AI agents. We'll build an intelligent agent that can remember conversations, learn from interactions, and make decisions based on historical data stored in Oracle Database.

## Overview

- **Cloud Platform**: Google Cloud Platform (GCP)
- **Database**: Oracle Autonomous Database@Google Cloud
- **AI Framework**: LangChain with Google Vertex AI
- **Memory Store**: Oracle Database (conversation history, agent state, knowledge base)
- **Use Cases**: Conversational AI, multi-turn dialogues, stateful agents, knowledge retrieval

## Architecture

```
┌─────────────────────────────┐
│   Google Colab / Local      │
│   (AI Agent Runtime)        │
└──────────────┬──────────────┘
               │
               │ (Network connection)
               │
┌──────────────▼──────────────┐
│   Google Cloud Network      │
│   (Private/Public VPC)      │
└──────────────┬──────────────┘
               │
┌──────────────▼──────────────┐
│  Oracle Autonomous DB       │
│  @Google Cloud              │
│  ├─ Conversation History    │
│  ├─ Agent Memory State      │
│  ├─ Knowledge Base          │
│  ├─ User Profiles           │
│  └─ Interaction Logs        │
└─────────────────────────────┘
```

## Prerequisites

- Google Cloud Platform account with billing enabled
- Oracle Autonomous Database@Google Cloud instance running
- Database credentials (host, port, username, password, service name)
- Google Colab notebook (or Jupyter Notebook locally)
- Basic familiarity with Python and AI agents

## What You'll Learn

1. Authenticate with Google Cloud Platform
2. Connect to Oracle Autonomous Database from Google Colab
3. Set up LangChain with memory persistence in Oracle
4. Create an AI agent with oracle agent memory
5. Store and retrieve conversation history
6. Query agent memory for context and decision making
7. Scale agent interactions across multiple conversations
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 1: Install Required Libraries and Dependencies
</VSCode.Cell>
<VSCode.Cell language="python">
# Install required packages for Google Colab
import subprocess
import sys

packages = [
    'google-cloud-storage',
    'google-auth',
    'google-auth-httplib2',
    'google-cloud-vertexai',
    'google-cloud-secret-manager',
    'cx_Oracle>=8.0',
    'langchain>=0.1.0',
    'langchain-community',
    'langchain-google-vertexai',
    'pandas',
    'numpy',
    'python-dotenv',
    'SQLAlchemy>=2.0',
    'openai',  # For GPT models if using OpenAI
]

print("Installing required packages...")
for package in packages:
    print(f"  Installing {package}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("\n✓ All packages installed successfully!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 2: Google Cloud Authentication and Setup
</VSCode.Cell>
<VSCode.Cell language="python">
# Google Cloud Authentication
from google.colab import auth
import google.auth
from google.cloud import secretmanager

# Authenticate with Google Cloud
print("Authenticating with Google Cloud...")
auth.authenticate_user()
print("✓ Authentication successful!")

# Get current project
try:
    _, project_id = google.auth.default()
    print(f"Current GCP Project: {project_id}")
except Exception as e:
    print(f"Warning: Could not retrieve project ID - {e}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 3: Oracle Database Connection Setup

### Configuration

Configure your Oracle Database connection parameters below:
</VSCode.Cell>
<VSCode.Cell language="python">
import os
from dotenv import load_dotenv

# =============================================================================
# CONFIGURATION: Update these values with your Oracle Database details
# =============================================================================

# Option 1: Direct configuration (for testing)
ORACLE_HOST = "YOUR_ORACLE_DB_HOST"  # e.g., "oracle-db.c.xyz.internal"
ORACLE_PORT = 1521
ORACLE_USER = "YOUR_DB_USER"  # e.g., "appuser"
ORACLE_PASSWORD = "YOUR_DB_PASSWORD"
ORACLE_SERVICE_NAME = "YOUR_SERVICE_NAME"  # e.g., "myappdb_high"

# Option 2: Load from environment variables (secure method)
# Uncomment below if using environment variables or Google Secret Manager
# ORACLE_HOST = os.getenv("ORACLE_HOST", "")
# ORACLE_USER = os.getenv("ORACLE_USER", "")
# ORACLE_PASSWORD = os.getenv("ORACLE_PASSWORD", "")
# ORACLE_SERVICE_NAME = os.getenv("ORACLE_SERVICE_NAME", "")

# For Google Secret Manager (recommended for production)
def get_secret_from_gcp(secret_id, version_id="latest"):
    """Retrieve secret from Google Cloud Secret Manager"""
    try:
        client = secretmanager.SecretManagerServiceClient()
        _, project_id = google.auth.default()
        name = client.secret_version_path(project_id, secret_id, version_id)
        response = client.access_secret_version(request={"name": name})
        return response.payload.data.decode("UTF-8")
    except Exception as e:
        print(f"Error retrieving secret: {e}")
        return None

# Uncomment to use Google Secret Manager
# ORACLE_PASSWORD = get_secret_from_gcp("oracle-db-password")

print("=" * 60)
print("Oracle Database Configuration")
print("=" * 60)
print(f"Host: {ORACLE_HOST}")
print(f"Port: {ORACLE_PORT}")
print(f"User: {ORACLE_USER}")
print(f"Service: {ORACLE_SERVICE_NAME}")
print("=" * 60)
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 4: Test Oracle Database Connection
</VSCode.Cell>
<VSCode.Cell language="python">
import cx_Oracle

def test_oracle_connection():
    """Test connection to Oracle Database"""
    try:
        # Create DSN
        dsn = cx_Oracle.makedsn(
            host=ORACLE_HOST,
            port=ORACLE_PORT,
            service_name=ORACLE_SERVICE_NAME
        )
        
        # Connect
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=dsn
        )
        
        print("✓ Successfully connected to Oracle Database!")
        
        # Get database info
        cursor = connection.cursor()
        cursor.execute("SELECT BANNER FROM v$version WHERE ROWNUM=1")
        version = cursor.fetchone()
        print(f"Database Version: {version[0]}")
        
        cursor.close()
        connection.close()
        return True
        
    except Exception as e:
        print(f"✗ Connection failed: {e}")
        return False

# Test the connection
print("Testing Oracle Database connection...")
connection_successful = test_oracle_connection()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 5: Create Agent Memory Tables in Oracle

Initialize the database schema for storing AI agent memory:
</VSCode.Cell>
<VSCode.Cell language="python">
def create_memory_schema():
    """Create necessary tables for AI agent memory in Oracle Database"""
    
    try:
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, ORACLE_SERVICE_NAME)
        )
        cursor = connection.cursor()
        
        # Drop existing tables (for clean restart)
        print("Creating/Recreating agent memory schema...")
        
        # Table 1: Conversation History
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS ai_agent_conversations (
                conversation_id VARCHAR2(100) PRIMARY KEY,
                agent_id VARCHAR2(100) NOT NULL,
                user_id VARCHAR2(100) NOT NULL,
                created_at TIMESTAMP DEFAULT SYSTIMESTAMP,
                updated_at TIMESTAMP DEFAULT SYSTIMESTAMP,
                status VARCHAR2(50) DEFAULT 'active'
            )
        """)
        print("✓ Created ai_agent_conversations table")
        
        # Table 2: Message History
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS ai_agent_messages (
                message_id VARCHAR2(100) PRIMARY KEY,
                conversation_id VARCHAR2(100) NOT NULL,
                role VARCHAR2(50) NOT NULL,
                content CLOB NOT NULL,
                timestamp TIMESTAMP DEFAULT SYSTIMESTAMP,
                metadata CLOB,
                FOREIGN KEY (conversation_id) REFERENCES ai_agent_conversations(conversation_id)
            )
        """)
        print("✓ Created ai_agent_messages table")
        
        # Table 3: Agent Memory State
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS ai_agent_memory (
                memory_id VARCHAR2(100) PRIMARY KEY,
                conversation_id VARCHAR2(100) NOT NULL,
                memory_type VARCHAR2(50) NOT NULL,
                key VARCHAR2(200),
                value CLOB,
                created_at TIMESTAMP DEFAULT SYSTIMESTAMP,
                expires_at TIMESTAMP,
                FOREIGN KEY (conversation_id) REFERENCES ai_agent_conversations(conversation_id)
            )
        """)
        print("✓ Created ai_agent_memory table")
        
        # Table 4: Knowledge Base
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS ai_knowledge_base (
                doc_id VARCHAR2(100) PRIMARY KEY,
                title VARCHAR2(500),
                content CLOB,
                embedding_vector CLOB,
                category VARCHAR2(100),
                created_at TIMESTAMP DEFAULT SYSTIMESTAMP,
                updated_at TIMESTAMP DEFAULT SYSTIMESTAMP
            )
        """)
        print("✓ Created ai_knowledge_base table")
        
        # Table 5: Agent Interactions Log
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS ai_agent_interactions (
                interaction_id VARCHAR2(100) PRIMARY KEY,
                conversation_id VARCHAR2(100),
                agent_id VARCHAR2(100),
                interaction_type VARCHAR2(100),
                input_text CLOB,
                output_text CLOB,
                decision_made VARCHAR2(500),
                confidence_score NUMBER(3,2),
                execution_time_ms NUMBER,
                created_at TIMESTAMP DEFAULT SYSTIMESTAMP,
                FOREIGN KEY (conversation_id) REFERENCES ai_agent_conversations(conversation_id)
            )
        """)
        print("✓ Created ai_agent_interactions table")
        
        connection.commit()
        cursor.close()
        connection.close()
        
        print("\n✓ Schema creation successful!")
        return True
        
    except Exception as e:
        print(f"✗ Error creating schema: {e}")
        return False

# Create the schema
if connection_successful:
    create_memory_schema()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 6: Set Up LangChain Oracle Memory Integration
</VSCode.Cell>
<VSCode.Cell language="python">
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.schema import BaseMemory, BaseChatMessageHistory
from langchain.schema import HumanMessage, AIMessage, BaseMessage
from typing import Any, Dict, List
import json
import uuid
from datetime import datetime
import cx_Oracle

class OracleChatMessageHistory(BaseChatMessageHistory):
    """Oracle-backed chat message history for LangChain"""
    
    def __init__(self, conversation_id: str, db_config: Dict[str, Any]):
        self.conversation_id = conversation_id
        self.db_config = db_config
    
    def add_message(self, message: BaseMessage) -> None:
        """Add a message to the chat history"""
        try:
            connection = cx_Oracle.connect(
                user=self.db_config['user'],
                password=self.db_config['password'],
                dsn=cx_Oracle.makedsn(
                    self.db_config['host'],
                    self.db_config['port'],
                    self.db_config['service_name']
                )
            )
            cursor = connection.cursor()
            
            message_id = str(uuid.uuid4())
            role = "human" if isinstance(message, HumanMessage) else "ai"
            content = message.content
            
            cursor.execute("""
                INSERT INTO ai_agent_messages 
                (message_id, conversation_id, role, content, timestamp)
                VALUES (:1, :2, :3, :4, SYSTIMESTAMP)
            """, (message_id, self.conversation_id, role, content))
            
            connection.commit()
            cursor.close()
            connection.close()
            
        except Exception as e:
            print(f"Error adding message: {e}")
    
    def get_messages(self) -> List[BaseMessage]:
        """Get all messages from the conversation"""
        try:
            connection = cx_Oracle.connect(
                user=self.db_config['user'],
                password=self.db_config['password'],
                dsn=cx_Oracle.makedsn(
                    self.db_config['host'],
                    self.db_config['port'],
                    self.db_config['service_name']
                )
            )
            cursor = connection.cursor()
            
            cursor.execute("""
                SELECT role, content FROM ai_agent_messages
                WHERE conversation_id = :1
                ORDER BY timestamp ASC
            """, (self.conversation_id,))
            
            messages = []
            for role, content in cursor.fetchall():
                if role == "human":
                    messages.append(HumanMessage(content=content))
                else:
                    messages.append(AIMessage(content=content))
            
            cursor.close()
            connection.close()
            return messages
            
        except Exception as e:
            print(f"Error retrieving messages: {e}")
            return []

class OracleAgentMemory(ConversationBufferMemory):
    """Oracle-backed memory for AI agents"""
    
    def __init__(self, conversation_id: str, db_config: Dict[str, Any], **kwargs):
        self.conversation_id = conversation_id or str(uuid.uuid4())
        self.db_config = db_config
        
        chat_histories = OracleChatMessageHistory(self.conversation_id, db_config)
        super().__init__(chat_memory=chat_histories, **kwargs)
    
    def save_memory_key(self, key: str, value: Any, memory_type: str = "general") -> None:
        """Save a memory key-value pair to Oracle"""
        try:
            connection = cx_Oracle.connect(
                user=self.db_config['user'],
                password=self.db_config['password'],
                dsn=cx_Oracle.makedsn(
                    self.db_config['host'],
                    self.db_config['port'],
                    self.db_config['service_name']
                )
            )
            cursor = connection.cursor()
            
            memory_id = str(uuid.uuid4())
            value_str = json.dumps(value) if not isinstance(value, str) else value
            
            cursor.execute("""
                INSERT INTO ai_agent_memory 
                (memory_id, conversation_id, memory_type, key, value, created_at)
                VALUES (:1, :2, :3, :4, :5, SYSTIMESTAMP)
            """, (memory_id, self.conversation_id, memory_type, key, value_str))
            
            connection.commit()
            cursor.close()
            connection.close()
            
        except Exception as e:
            print(f"Error saving memory key: {e}")
    
    def get_memory_key(self, key: str) -> Any:
        """Retrieve a memory key from Oracle"""
        try:
            connection = cx_Oracle.connect(
                user=self.db_config['user'],
                password=self.db_config['password'],
                dsn=cx_Oracle.makedsn(
                    self.db_config['host'],
                    self.db_config['port'],
                    self.db_config['service_name']
                )
            )
            cursor = connection.cursor()
            
            cursor.execute("""
                SELECT value FROM ai_agent_memory
                WHERE conversation_id = :1 AND key = :2
                ORDER BY created_at DESC
                FETCH FIRST 1 ROWS ONLY
            """, (self.conversation_id, key))
            
            result = cursor.fetchone()
            cursor.close()
            connection.close()
            
            if result:
                value_str = result[0]
                try:
                    return json.loads(value_str)
                except:
                    return value_str
            return None
            
        except Exception as e:
            print(f"Error retrieving memory key: {e}")
            return None

print("✓ OracleChatMessageHistory and OracleAgentMemory classes created successfully!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 7: Initialize AI Agent with Oracle Memory

Create an AI agent powered by Google Vertex AI with Oracle Database as memory backend:
</VSCode.Cell>
<VSCode.Cell language="python">
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatVertexAI
from langchain.tools import tool
from langchain.callbacks import StdOutCallbackHandler

# Initialize Vertex AI LLM
print("Initializing Google Vertex AI LLM...")
llm = ChatVertexAI(
    model_name="gemini-pro",
    temperature=0.7,
    max_output_tokens=1024
)
print("✓ Vertex AI LLM initialized")

# Initialize Oracle Memory
db_config = {
    'user': ORACLE_USER,
    'password': ORACLE_PASSWORD,
    'host': ORACLE_HOST,
    'port': ORACLE_PORT,
    'service_name': ORACLE_SERVICE_NAME
}

conversation_id = str(uuid.uuid4())
memory = OracleAgentMemory(
    conversation_id=conversation_id,
    db_config=db_config,
    memory_key="chat_history",
    human_prefix="User",
    ai_prefix="Agent",
    return_messages=True
)

print(f"✓ Oracle Memory initialized (Conversation ID: {conversation_id})")

# Define custom tools for the agent
@tool
def query_knowledge_base(query: str) -> str:
    """Query the knowledge base from Oracle Database"""
    try:
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, ORACLE_SERVICE_NAME)
        )
        cursor = connection.cursor()
        
        cursor.execute("""
            SELECT title, content FROM ai_knowledge_base
            WHERE LOWER(title) LIKE LOWER('%' || :1 || '%')
                OR LOWER(content) LIKE LOWER('%' || :1 || '%')
            FETCH FIRST 5 ROWS ONLY
        """, (query,))
        
        results = cursor.fetchall()
        cursor.close()
        connection.close()
        
        if not results:
            return "No matching documents found in knowledge base."
        
        response = "Found the following documents:\n"
        for i, (title, content) in enumerate(results, 1):
            response += f"\n{i}. {title}\n   {content[:200]}...\n"
        
        return response
        
    except Exception as e:
        return f"Error querying knowledge base: {e}"

@tool
def save_user_context(context_key: str, context_value: str) -> str:
    """Save user context to agent memory"""
    memory.save_memory_key(context_key, context_value, memory_type="user_context")
    return f"Saved context: {context_key}"

@tool
def retrieve_user_context(context_key: str) -> str:
    """Retrieve user context from agent memory"""
    value = memory.get_memory_key(context_key)
    return str(value) if value else "No context found"

# Define tools
tools = [
    Tool(
        name="Query Knowledge Base",
        func=query_knowledge_base,
        description="Search and retrieve information from the knowledge base stored in Oracle Database"
    ),
    Tool(
        name="Save User Context",
        func=save_user_context,
        description="Save user context or preferences to agent memory for future reference"
    ),
    Tool(
        name="Retrieve User Context",
        func=retrieve_user_context,
        description="Retrieve previously saved user context from agent memory"
    )
]

# Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    callbacks=[StdOutCallbackHandler()]
)

print("✓ AI Agent initialized with Oracle Memory backend!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 8: Interact with the AI Agent

Have multi-turn conversations with the agent that persist in Oracle Database:
</VSCode.Cell>
<VSCode.Cell language="python">
# First, let's add some sample data to the knowledge base
def add_sample_knowledge():
    """Add sample documents to the knowledge base"""
    try:
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, ORACLE_SERVICE_NAME)
        )
        cursor = connection.cursor()
        
        sample_docs = [
            (str(uuid.uuid4()), "Oracle Database Features", 
             "Oracle Database offers advanced features like AI/ML capabilities, JSON support, and vector search.", 
             "database"),
            (str(uuid.uuid4()), "Google Cloud Integration", 
             "Oracle Autonomous Database can be deployed on Google Cloud and integrates with GCP services.", 
             "cloud"),
            (str(uuid.uuid4()), "AI Agent Development", 
             "AI agents are intelligent systems that can perceive their environment and take actions to achieve goals.", 
             "ai"),
        ]
        
        for doc_id, title, content, category in sample_docs:
            cursor.execute("""
                INSERT INTO ai_knowledge_base 
                (doc_id, title, content, category, created_at, updated_at)
                VALUES (:1, :2, :3, :4, SYSTIMESTAMP, SYSTIMESTAMP)
            """, (doc_id, title, content, category))
        
        connection.commit()
        cursor.close()
        connection.close()
        print("✓ Sample knowledge base documents added")
        
    except Exception as e:
        print(f"Note: {e}")

add_sample_knowledge()

# Example interactions
print("\n" + "=" * 70)
print("MULTI-TURN AI AGENT CONVERSATION WITH ORACLE MEMORY")
print("=" * 70 + "\n")

# Conversation turns
conversations = [
    "Hello! I'm interested in learning about Oracle Database features.",
    "Can you save my preference for database topics?",
    "What can you tell me about Google Cloud Integration?",
    "Based on my preferences, what would you recommend for an enterprise application?"
]

for i, user_input in enumerate(conversations, 1):
    print(f"\n{'─' * 70}")
    print(f"Turn {i}")
    print(f"{'─' * 70}")
    print(f"\nUser: {user_input}")
    print("-" * 70)
    
    try:
        response = agent.run(user_input)
        print(f"\nAgent: {response}")
    except Exception as e:
        print(f"Error during agent execution: {e}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 9: Query Agent Memory and Conversation History
</VSCode.Cell>
<VSCode.Cell language="python">
def retrieve_conversation_history(conv_id: str):
    """Retrieve and display complete conversation history from Oracle"""
    try:
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, ORACLE_SERVICE_NAME)
        )
        cursor = connection.cursor()
        
        print("\n" + "=" * 70)
        print("CONVERSATION HISTORY FROM ORACLE DATABASE")
        print("=" * 70 + "\n")
        
        cursor.execute("""
            SELECT role, content, timestamp FROM ai_agent_messages
            WHERE conversation_id = :1
            ORDER BY timestamp ASC
        """, (conv_id,))
        
        messages = cursor.fetchall()
        
        for i, (role, content, timestamp) in enumerate(messages, 1):
            role_display = "👤 User" if role == "human" else "🤖 Agent"
            print(f"{i}. [{timestamp}] {role_display}:")
            print(f"   {content}\n")
        
        cursor.close()
        connection.close()
        
        print(f"Total messages in conversation: {len(messages)}")
        
    except Exception as e:
        print(f"Error retrieving conversation history: {e}")

def retrieve_agent_memory(conv_id: str):
    """Retrieve and display agent memory state from Oracle"""
    try:
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, ORACLE_SERVICE_NAME)
        )
        cursor = connection.cursor()
        
        print("\n" + "=" * 70)
        print("AGENT MEMORY STATE FROM ORACLE DATABASE")
        print("=" * 70 + "\n")
        
        cursor.execute("""
            SELECT memory_type, key, value, created_at FROM ai_agent_memory
            WHERE conversation_id = :1
            ORDER BY created_at DESC
        """, (conv_id,))
        
        memory_items = cursor.fetchall()
        
        if not memory_items:
            print("No memory items found.")
        else:
            for i, (mem_type, key, value, created_at) in enumerate(memory_items, 1):
                print(f"{i}. [{mem_type}] {key} = {value}")
                print(f"   Created: {created_at}\n")
        
        cursor.close()
        connection.close()
        
        print(f"Total memory items: {len(memory_items)}")
        
    except Exception as e:
        print(f"Error retrieving agent memory: {e}")

# Display results
retrieve_conversation_history(conversation_id)
retrieve_agent_memory(conversation_id)
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 10: Monitor Agent Interactions and Metrics
</VSCode.Cell>
<VSCode.Cell language="python">
def log_interaction(conversation_id: str, agent_id: str, interaction_type: str, 
                   input_text: str, output_text: str, decision_made: str = None, 
                   confidence_score: float = None, execution_time_ms: float = None):
    """Log agent interactions for monitoring and analysis"""
    try:
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, ORACLE_SERVICE_NAME)
        )
        cursor = connection.cursor()
        
        interaction_id = str(uuid.uuid4())
        
        cursor.execute("""
            INSERT INTO ai_agent_interactions 
            (interaction_id, conversation_id, agent_id, interaction_type, 
             input_text, output_text, decision_made, confidence_score, 
             execution_time_ms, created_at)
            VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, SYSTIMESTAMP)
        """, (interaction_id, conversation_id, agent_id, interaction_type,
              input_text, output_text, decision_made, confidence_score, execution_time_ms))
        
        connection.commit()
        cursor.close()
        connection.close()
        
        return interaction_id
        
    except Exception as e:
        print(f"Error logging interaction: {e}")
        return None

def get_agent_statistics(agent_id: str):
    """Retrieve agent performance statistics from Oracle"""
    try:
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, ORACLE_SERVICE_NAME)
        )
        cursor = connection.cursor()
        
        print("\n" + "=" * 70)
        print(f"AGENT STATISTICS FOR: {agent_id}")
        print("=" * 70 + "\n")
        
        # Total interactions
        cursor.execute("""
            SELECT COUNT(*) FROM ai_agent_interactions WHERE agent_id = :1
        """, (agent_id,))
        total_interactions = cursor.fetchone()[0]
        print(f"Total Interactions: {total_interactions}")
        
        # Average confidence score
        cursor.execute("""
            SELECT AVG(confidence_score) FROM ai_agent_interactions 
            WHERE agent_id = :1 AND confidence_score IS NOT NULL
        """, (agent_id,))
        avg_confidence = cursor.fetchone()[0]
        print(f"Average Confidence Score: {avg_confidence:.2%}" if avg_confidence else "N/A")
        
        # Average execution time
        cursor.execute("""
            SELECT AVG(execution_time_ms) FROM ai_agent_interactions 
            WHERE agent_id = :1 AND execution_time_ms IS NOT NULL
        """, (agent_id,))
        avg_execution_time = cursor.fetchone()[0]
        print(f"Average Execution Time: {avg_execution_time:.2f}ms" if avg_execution_time else "N/A")
        
        # Interaction type breakdown
        cursor.execute("""
            SELECT interaction_type, COUNT(*) 
            FROM ai_agent_interactions 
            WHERE agent_id = :1
            GROUP BY interaction_type
        """, (agent_id,))
        
        print("\nInteraction Type Breakdown:")
        for interaction_type, count in cursor.fetchall():
            print(f"  - {interaction_type}: {count}")
        
        cursor.close()
        connection.close()
        
    except Exception as e:
        print(f"Error retrieving statistics: {e}")

# Example: Get statistics for the agent
agent_id = "vertex-ai-agent-001"
get_agent_statistics(agent_id)
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 11: Advanced Features - Context-Aware Decision Making
</VSCode.Cell>
<VSCode.Cell language="python">
class ContextAwareAgent:
    """Agent that makes decisions based on historical context from Oracle"""
    
    def __init__(self, agent_id: str, conversation_id: str, db_config: Dict[str, Any]):
        self.agent_id = agent_id
        self.conversation_id = conversation_id
        self.db_config = db_config
    
    def get_user_history_summary(self):
        """Get summary of user's interaction history"""
        try:
            connection = cx_Oracle.connect(
                user=self.db_config['user'],
                password=self.db_config['password'],
                dsn=cx_Oracle.makedsn(
                    self.db_config['host'],
                    self.db_config['port'],
                    self.db_config['service_name']
                )
            )
            cursor = connection.cursor()
            
            cursor.execute("""
                SELECT 
                    COUNT(*) as total_messages,
                    COUNT(CASE WHEN role = 'human' THEN 1 END) as user_messages,
                    COUNT(CASE WHEN role = 'ai' THEN 1 END) as agent_messages
                FROM ai_agent_messages
                WHERE conversation_id = :1
            """, (self.conversation_id,))
            
            total_msgs, user_msgs, agent_msgs = cursor.fetchone()
            
            cursor.close()
            connection.close()
            
            summary = {
                'total_messages': total_msgs,
                'user_messages': user_msgs,
                'agent_messages': agent_msgs
            }
            
            return summary
            
        except Exception as e:
            print(f"Error getting history summary: {e}")
            return {}
    
    def get_conversation_sentiment_trend(self):
        """Analyze conversation sentiment trend"""
        summary = self.get_user_history_summary()
        
        print("\n" + "=" * 70)
        print("CONVERSATION ANALYSIS")
        print("=" * 70 + "\n")
        print(f"Total Messages: {summary.get('total_messages', 0)}")
        print(f"User Messages: {summary.get('user_messages', 0)}")
        print(f"Agent Messages: {summary.get('agent_messages', 0)}")
        
        if summary.get('total_messages', 0) > 0:
            user_ratio = (summary.get('user_messages', 0) / summary.get('total_messages', 1)) * 100
            print(f"User Activity Ratio: {user_ratio:.1f}%")

# Create context-aware agent instance
context_agent = ContextAwareAgent(agent_id, conversation_id, db_config)
context_agent.get_conversation_sentiment_trend()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 12: Cleanup and Best Practices
</VSCode.Cell>
<VSCode.Cell language="python">
def cleanup_old_conversations(days_old: int = 30):
    """Archive or clean up old conversations from Oracle"""
    try:
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, ORACLE_SERVICE_NAME)
        )
        cursor = connection.cursor()
        
        # Archive old conversations
        cursor.execute("""
            UPDATE ai_agent_conversations
            SET status = 'archived'
            WHERE created_at < TRUNC(SYSDATE) - :1
            AND status = 'active'
        """, (days_old,))
        
        archived_count = cursor.rowcount
        connection.commit()
        
        print(f"✓ Archived {archived_count} conversations older than {days_old} days")
        
        cursor.close()
        connection.close()
        
    except Exception as e:
        print(f"Error during cleanup: {e}")

def export_conversation_to_json(conversation_id: str):
    """Export conversation to JSON for analysis or backup"""
    try:
        connection = cx_Oracle.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, ORACLE_SERVICE_NAME)
        )
        cursor = connection.cursor()
        
        # Get conversation metadata
        cursor.execute("""
            SELECT conversation_id, agent_id, user_id, created_at 
            FROM ai_agent_conversations
            WHERE conversation_id = :1
        """, (conversation_id,))
        
        conv_data = cursor.fetchone()
        
        if not conv_data:
            print("Conversation not found")
            return None
        
        # Get messages
        cursor.execute("""
            SELECT role, content, timestamp FROM ai_agent_messages
            WHERE conversation_id = :1
            ORDER BY timestamp ASC
        """, (conversation_id,))
        
        messages = [
            {"role": role, "content": content, "timestamp": str(timestamp)}
            for role, content, timestamp in cursor.fetchall()
        ]
        
        export_data = {
            'conversation_id': conv_data[0],
            'agent_id': conv_data[1],
            'user_id': conv_data[2],
            'created_at': str(conv_data[3]),
            'messages': messages
        }
        
        cursor.close()
        connection.close()
        
        return export_data
        
    except Exception as e:
        print(f"Error exporting conversation: {e}")
        return None

# Example cleanup (with safety check for demo)
print("Cleanup capabilities available:")
print("- cleanup_old_conversations(days_old=30) - Archive old conversations")
print("- export_conversation_to_json(conversation_id) - Export for analysis")

# Export current conversation
exported = export_conversation_to_json(conversation_id)
if exported:
    print(f"\n✓ Conversation exported: {len(exported['messages'])} messages")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 13: Best Practices and Production Deployment

### Security Best Practices

1. **Credential Management**
   - Use Google Secret Manager for storing database credentials
   - Never hardcode passwords in notebooks
   - Rotate credentials regularly

2. **Network Security**
   - Deploy Oracle Database in private VPC
   - Use Private Service Connection for Google Colab
   - Implement firewall rules limiting access

3. **Data Protection**
   - Enable Oracle Database encryption at rest
   - Use TLS for all database connections
   - Implement row-level security (RLS) for multi-tenant scenarios

### Performance Optimization

1. **Connection Pooling**
   - Use connection pooling with SQLAlchemy
   - Set appropriate pool size and timeout

2. **Query Optimization**
   - Create indexes on frequently queried columns (conversation_id, agent_id)
   - Use pagination for large result sets

3. **Batch Operations**
   - Batch message inserts during high-volume conversations
   - Use bulk operations for log archival

### Monitoring and Logging

- Enable Oracle Database audit trail
- Set up monitoring dashboards in Google Cloud Console
- Implement alerting for agent failures
- Track agent performance metrics

### Cost Optimization

- Use Always Free tier for development/testing
- Implement data retention policies
- Archive old conversations to Cloud Storage
- Monitor database resource utilization
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 14: Troubleshooting Common Issues

### Issue 1: Cannot Connect to Oracle Database

**Symptoms:** Connection timeout or "ORA-12514 TNS:listener does not currently know of service requested"

**Solution:**
```python
# Verify connection parameters
print(f"Host: {ORACLE_HOST}")
print(f"Port: {ORACLE_PORT}")
print(f"Service: {ORACLE_SERVICE_NAME}")

# Test network connectivity
import socket
try:
    socket.create_connection((ORACLE_HOST, ORACLE_PORT), timeout=5)
    print("✓ Network connectivity OK")
except:
    print("✗ Cannot reach Oracle Database host")
```

### Issue 2: Memory Tables Not Found

**Symptoms:** "ORA-00942: table or view does not exist"

**Solution:**
```python
# Recreate schema
create_memory_schema()
```

### Issue 3: Slow Query Performance

**Symptoms:** Long query execution times

**Solution:**
```python
# Create indexes
connection = cx_Oracle.connect(...)
cursor = connection.cursor()

cursor.execute("""
    CREATE INDEX idx_conv_id ON ai_agent_messages(conversation_id)
""")
cursor.execute("""
    CREATE INDEX idx_agent_id ON ai_agent_interactions(agent_id)
""")
connection.commit()
```

### Issue 4: Memory Quota Exceeded in Colab

**Solution:**
- Use smaller batch sizes
- Stream large result sets
- Archive old conversations regularly
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Section 15: Next Steps and Advanced Topics

### Recommended Next Steps

1. **Implement Semantic Search**
   - Store embeddings in Oracle Vector data type
   - Implement similarity search for knowledge base

2. **Add LLM Fine-tuning**
   - Collect agent interaction data
   - Fine-tune models on domain-specific data

3. **Build Multi-Agent System**
   - Create specialized agents for different tasks
   - Implement agent coordination and communication

4. **Production Deployment**
   - Deploy as Google Cloud Run service
   - Set up CI/CD pipeline
   - Implement monitoring and alerting

### References and Resources

- [Oracle Autonomous Database Documentation](https://docs.oracle.com/en/database/)
- [LangChain Documentation](https://python.langchain.com/docs/)
- [Google Vertex AI Documentation](https://cloud.google.com/vertex-ai/docs)
- [Google Cloud SQL Proxy](https://cloud.google.com/sql/docs/mysql/sql-proxy)
- [Oracle AI/ML Features](https://www.oracle.com/ai/)

### Community and Support

- [Oracle Community Forums](https://community.oracle.com/)
- [LangChain Discord](https://discord.gg/6adMQxSpJS)
- [Google Cloud Support](https://cloud.google.com/support)
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Summary

You've successfully created an AI agent with Oracle Database as memory backend! 

**Key Accomplishments:**
- ✓ Connected to Oracle Autonomous Database on Google Cloud
- ✓ Created persistent memory schema in Oracle
- ✓ Implemented LangChain integration for AI agents
- ✓ Built context-aware decision making
- ✓ Stored and retrieved conversation history
- ✓ Monitored agent performance metrics

**Architecture Benefits:**
- Scalable: Support thousands of concurrent conversations
- Persistent: Never lose conversation history
- Secure: Enterprise-grade Oracle Database security
- Observable: Track all agent interactions
- Extensible: Add custom tools and integrations

For production deployment, consider containerizing this notebook as a Google Cloud Run service combined with Cloud Scheduler for periodic cleanup tasks.
</VSCode.Cell>
````